In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
app_test = pd.read_csv('home-credit-default-risk/application_test.csv')
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
bureau = pd.read_csv('home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('home-credit-default-risk/bureau_balance.csv')
pos_cash_balance = pd.read_csv('home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('home-credit-default-risk/credit_card_balance.csv')
installments_payments = pd.read_csv('home-credit-default-risk/installments_payments.csv')
previous_application = pd.read_csv('home-credit-default-risk/previous_application.csv')

In [4]:
columns_to_keep = ['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_INCOME_TYPE',
                   'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'FLAG_EMP_PHONE',
                   'FLAG_WORK_PHONE', 'REGION_RATING_CLIENT',
                   'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY',
                   'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_6',
                   'ORGANIZATION_TYPE', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
                   'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
                   'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
                   'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_2', 'OBS_30_CNT_SOCIAL_CIRCLE',
                   'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'TOTAL_CREDITS',
                   'ACTIVE_CREDITS', 'AVG_DAYS_CREDIT', 'CLOSED_CREDITS',
                   'ACTIVE_CREDIT_RATIO', 'TOTAL_MONTHS_DPD', 'AVG_CREDIT_LIMIT_USAGE',
                   'ACTIVE_CREDIT_CARDS', 'LATE_PAYMENTS_COUNT', 'AVG_LATE_DAYS',
                   'LATE_PAYMENT_RATIO', 'PREV_TOTAL_APPLICATIONS', 'PREV_APPROVED_RATIO',
                   'PREV_AVG_AMT_CREDIT', 'YEARS_BIRTH', 'CNT_FAM_MEMBERS',
                   'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'CNT_CHILDREN']

def filter_columns(data, columns_to_keep):
    """
    Memfilter kolom yang ada dalam data berdasarkan daftar kolom yang diberikan.
    
    Parameters:
    data (pd.DataFrame): DataFrame yang berisi data yang akan diproses.
    columns_to_keep (list): Daftar kolom yang ingin dipilih dari data.
    
    Returns:
    pd.DataFrame: DataFrame yang hanya berisi kolom yang ada di columns_to_keep.
    """

    filtered_columns = [col for col in columns_to_keep if col in data.columns]
    missing_columns = [col for col in columns_to_keep if col not in data.columns]
    
    filtered_data = data[filtered_columns]

    if missing_columns:
        print(f"Tidak ada kolom berikut dalam data: {', '.join(missing_columns)}")
    else:
        print("Semua kolom tersedia dalam data.")

    return filtered_data

app_test_new = filter_columns(app_test, columns_to_keep)
app_test_new.head() 


Tidak ada kolom berikut dalam data: TOTAL_CREDITS, ACTIVE_CREDITS, AVG_DAYS_CREDIT, CLOSED_CREDITS, ACTIVE_CREDIT_RATIO, TOTAL_MONTHS_DPD, AVG_CREDIT_LIMIT_USAGE, ACTIVE_CREDIT_CARDS, LATE_PAYMENTS_COUNT, AVG_LATE_DAYS, LATE_PAYMENT_RATIO, PREV_TOTAL_APPLICATIONS, PREV_APPROVED_RATIO, PREV_AVG_AMT_CREDIT, YEARS_BIRTH


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,DAYS_ID_PUBLISH,HOUR_APPR_PROCESS_START,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,CNT_FAM_MEMBERS,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,CNT_CHILDREN
0,100001,Cash loans,F,Working,Higher education,House / apartment,1,0,2,2,...,-812,18,0.789654,0.0,0.0,-1740.0,2.0,0.0,0.0,0
1,100005,Cash loans,M,Working,Secondary / secondary special,House / apartment,1,0,2,2,...,-1623,9,0.291656,0.0,0.0,0.0,2.0,0.0,0.0,0
2,100013,Cash loans,M,Working,Higher education,House / apartment,1,0,2,2,...,-3503,14,0.699787,0.0,0.0,-856.0,2.0,0.0,0.0,0
3,100028,Cash loans,F,Working,Secondary / secondary special,House / apartment,1,0,2,2,...,-4208,11,0.509677,0.0,0.0,-1805.0,4.0,0.0,0.0,2
4,100038,Cash loans,M,Working,Secondary / secondary special,House / apartment,1,1,2,2,...,-4262,5,0.425687,0.0,0.0,-821.0,3.0,0.0,0.0,1


In [5]:
total_credits_per_client = bureau.groupby('SK_ID_CURR').size().reset_index(name='TOTAL_CREDITS')

active_credits_per_client = bureau[bureau['CREDIT_ACTIVE'] == "Active"] \
    .groupby('SK_ID_CURR').size().reset_index(name='ACTIVE_CREDITS')

avg_days_credit_per_client = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'] \
    .mean().reset_index(name='AVG_DAYS_CREDIT')

closed_credits_per_client = bureau[bureau['CREDIT_ACTIVE'] == "Closed"] \
    .groupby('SK_ID_CURR').size().reset_index(name='CLOSED_CREDITS')

credit_ratios = total_credits_per_client.merge(active_credits_per_client, on='SK_ID_CURR', how='left')
credit_ratios['ACTIVE_CREDIT_RATIO'] = credit_ratios['ACTIVE_CREDITS'] / credit_ratios['TOTAL_CREDITS']
credit_ratios['ACTIVE_CREDIT_RATIO'] = credit_ratios['ACTIVE_CREDIT_RATIO'].fillna(0) 

result = total_credits_per_client \
    .merge(active_credits_per_client, on='SK_ID_CURR', how='left') \
    .merge(avg_days_credit_per_client, on='SK_ID_CURR', how='left') \
    .merge(closed_credits_per_client, on='SK_ID_CURR', how='left') \
    .merge(credit_ratios[['SK_ID_CURR', 'ACTIVE_CREDIT_RATIO']], on='SK_ID_CURR', how='left')

result.fillna(0, inplace=True)

In [6]:
result.head()

,SK_ID_CURR,TOTAL_CREDITS,ACTIVE_CREDITS,AVG_DAYS_CREDIT,CLOSED_CREDITS,ACTIVE_CREDIT_RATIO
0,100001,7,3.0,-735.000000,4.0,0.428571
1,100002,8,2.0,-874.000000,6.0,0.250000
2,100003,4,1.0,-1400.750000,3.0,0.250000
3,100004,2,0.0,-867.000000,2.0,0.000000
4,100005,3,2.0,-190.666667,1.0,0.666667


In [7]:
app_test_new = app_test_new.merge(result, on='SK_ID_CURR')

In [8]:
total_months_dpd = credit_card_balance[credit_card_balance['SK_DPD'] > 0] \
    .groupby('SK_ID_CURR').size().reset_index(name='TOTAL_MONTHS_DPD')

credit_card_balance['CREDIT_LIMIT_USAGE'] = credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
average_credit_limit_usage = credit_card_balance.groupby('SK_ID_CURR')['CREDIT_LIMIT_USAGE'] \
    .mean().reset_index(name='AVG_CREDIT_LIMIT_USAGE')

active_credit_cards = credit_card_balance[credit_card_balance['NAME_CONTRACT_STATUS'] == "Active"] \
    .groupby('SK_ID_CURR').size().reset_index(name='ACTIVE_CREDIT_CARDS')

result = total_months_dpd \
    .merge(average_credit_limit_usage, on='SK_ID_CURR', how='left') \
    .merge(active_credit_cards, on='SK_ID_CURR', how='left')
result.fillna(0, inplace=True)

result.head()

,SK_ID_CURR,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS
0,100013,1,0.115301,96.0
1,100042,2,0.370624,84.0
2,100067,3,0.604061,87.0
3,100082,10,0.144965,88.0
4,100086,19,0.374557,84.0


In [9]:
app_test_new = app_test_new.merge(result, on='SK_ID_CURR')

In [10]:
filter_columns(app_test_new, columns_to_keep)

Tidak ada kolom berikut dalam data: LATE_PAYMENTS_COUNT, AVG_LATE_DAYS, LATE_PAYMENT_RATIO, PREV_TOTAL_APPLICATIONS, PREV_APPROVED_RATIO, PREV_AVG_AMT_CREDIT, YEARS_BIRTH


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,AVG_DAYS_CREDIT,CLOSED_CREDITS,ACTIVE_CREDIT_RATIO,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS,CNT_FAM_MEMBERS,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,CNT_CHILDREN
0,100013,Cash loans,M,Working,Higher education,House / apartment,1,0,2,2,...,-1737.500000,4.0,0.000000,1,0.115301,96.0,2.0,0.0,0.0,0
1,100042,Cash loans,F,State servant,Secondary / secondary special,House / apartment,1,0,2,2,...,-1325.500000,6.0,0.571429,2,0.370624,84.0,2.0,0.0,0.0,0
2,100067,Cash loans,F,Working,Higher education,House / apartment,1,1,2,2,...,-579.529412,12.0,0.294118,3,0.604061,87.0,3.0,0.0,0.0,1
3,100107,Cash loans,M,Working,Secondary / secondary special,House / apartment,1,0,1,1,...,-17.000000,0.0,1.000000,1,0.665672,11.0,2.0,0.0,0.0,0
4,100109,Cash loans,F,Working,Secondary / secondary special,House / apartment,1,1,2,2,...,-1155.615385,10.0,0.230769,3,0.283998,72.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,455378,Cash loans,F,State servant,Secondary / secondary special,House / apartment,1,0,2,2,...,-1492.250000,12.0,0.250000,1,0.264344,96.0,2.0,0.0,0.0,0
2484,455436,Cash loans,F,Working,Lower secondary,House / apartment,1,1,1,1,...,-1548.666667,3.0,0.000000,4,0.932081,96.0,2.0,0.0,0.0,0
2485,455648,Cash loans,F,Commercial associate,Higher education,House / apartment,1,0,2,2,...,-1135.500000,9.0,0.100000,1,0.199751,96.0,3.0,1.0,1.0,1
2486,455863,Cash loans,F,Pensioner,Higher education,House / apartment,0,0,2,2,...,-1578.363636,7.0,0.363636,1,0.436418,38.0,2.0,0.0,0.0,0


In [11]:
late_payments = installments_payments[installments_payments['DAYS_ENTRY_PAYMENT'] > installments_payments['DAYS_INSTALMENT']]
late_payments_count = late_payments.groupby('SK_ID_CURR').size().reset_index(name='LATE_PAYMENTS_COUNT')

late_payments['LATE_DAYS'] = late_payments['DAYS_ENTRY_PAYMENT'] - late_payments['DAYS_INSTALMENT']
avg_late_days = late_payments.groupby('SK_ID_CURR')['LATE_DAYS'] \
    .mean().reset_index(name='AVG_LATE_DAYS')

total_payments = installments_payments.groupby('SK_ID_CURR').size().reset_index(name='TOTAL_PAYMENTS')
late_payment_ratio = late_payments_count.merge(total_payments, on='SK_ID_CURR', how='left')
late_payment_ratio['LATE_PAYMENT_RATIO'] = late_payment_ratio['LATE_PAYMENTS_COUNT'] / late_payment_ratio['TOTAL_PAYMENTS']

result = late_payments_count \
    .merge(avg_late_days, on='SK_ID_CURR', how='left') \
    .merge(late_payment_ratio[['SK_ID_CURR', 'LATE_PAYMENT_RATIO']], on='SK_ID_CURR', how='left')

result.fillna(0, inplace=True)

result.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_2584\2719364529.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  late_payments['LATE_DAYS'] = late_payments['DAYS_ENTRY_PAYMENT'] - late_payments['DAYS_INSTALMENT']


,SK_ID_CURR,LATE_PAYMENTS_COUNT,AVG_LATE_DAYS,LATE_PAYMENT_RATIO
0,100001,1,11.0000,0.142857
1,100005,1,1.0000,0.111111
2,100007,16,3.9375,0.242424
3,100008,1,1317.0000,0.028571
4,100009,1,7.0000,0.019608


In [12]:
app_test_new = app_test_new.merge(result, on='SK_ID_CURR')
app_test_new.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,ACTIVE_CREDITS,AVG_DAYS_CREDIT,CLOSED_CREDITS,ACTIVE_CREDIT_RATIO,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS,LATE_PAYMENTS_COUNT,AVG_LATE_DAYS,LATE_PAYMENT_RATIO
0,100013,Cash loans,M,Working,Higher education,House / apartment,1,0,2,2,...,0.0,-1737.500000,4.0,0.000000,1,0.115301,96.0,11,7.636364,0.070968
1,100042,Cash loans,F,State servant,Secondary / secondary special,House / apartment,1,0,2,2,...,8.0,-1325.500000,6.0,0.571429,2,0.370624,84.0,4,8.750000,0.021978
2,100067,Cash loans,F,Working,Higher education,House / apartment,1,1,2,2,...,5.0,-579.529412,12.0,0.294118,3,0.604061,87.0,6,3.333333,0.037267
3,100107,Cash loans,M,Working,Secondary / secondary special,House / apartment,1,0,1,1,...,1.0,-17.000000,0.0,1.000000,1,0.665672,11.0,3,17.333333,0.075000
4,100109,Cash loans,F,Working,Secondary / secondary special,House / apartment,1,1,2,2,...,3.0,-1155.615385,10.0,0.230769,3,0.283998,72.0,3,5.000000,0.034091


In [13]:
filter_columns(app_test_new, columns_to_keep)


Tidak ada kolom berikut dalam data: PREV_TOTAL_APPLICATIONS, PREV_APPROVED_RATIO, PREV_AVG_AMT_CREDIT, YEARS_BIRTH


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS,LATE_PAYMENTS_COUNT,AVG_LATE_DAYS,LATE_PAYMENT_RATIO,CNT_FAM_MEMBERS,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,CNT_CHILDREN
0,100013,Cash loans,M,Working,Higher education,House / apartment,1,0,2,2,...,1,0.115301,96.0,11,7.636364,0.070968,2.0,0.0,0.0,0
1,100042,Cash loans,F,State servant,Secondary / secondary special,House / apartment,1,0,2,2,...,2,0.370624,84.0,4,8.750000,0.021978,2.0,0.0,0.0,0
2,100067,Cash loans,F,Working,Higher education,House / apartment,1,1,2,2,...,3,0.604061,87.0,6,3.333333,0.037267,3.0,0.0,0.0,1
3,100107,Cash loans,M,Working,Secondary / secondary special,House / apartment,1,0,1,1,...,1,0.665672,11.0,3,17.333333,0.075000,2.0,0.0,0.0,0
4,100109,Cash loans,F,Working,Secondary / secondary special,House / apartment,1,1,2,2,...,3,0.283998,72.0,3,5.000000,0.034091,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,455378,Cash loans,F,State servant,Secondary / secondary special,House / apartment,1,0,2,2,...,1,0.264344,96.0,4,4.250000,0.030769,2.0,0.0,0.0,0
2454,455436,Cash loans,F,Working,Lower secondary,House / apartment,1,1,1,1,...,4,0.932081,96.0,5,8.000000,0.038462,2.0,0.0,0.0,0
2455,455648,Cash loans,F,Commercial associate,Higher education,House / apartment,1,0,2,2,...,1,0.199751,96.0,5,7.600000,0.027624,3.0,1.0,1.0,1
2456,455863,Cash loans,F,Pensioner,Higher education,House / apartment,0,0,2,2,...,1,0.436418,38.0,5,9.200000,0.054348,2.0,0.0,0.0,0


In [14]:
PREV_TOTAL_APPLICATIONS = previous_application.groupby('SK_ID_CURR').size().reset_index(name='PREV_TOTAL_APPLICATIONS')

approved_apps = previous_application[previous_application['NAME_CONTRACT_STATUS'] == "Approved"]
PREV_APPROVED_RATIO = approved_apps.groupby('SK_ID_CURR').size().reset_index(name='PREV_APPROVED_COUNT')
PREV_APPROVED_RATIO = PREV_APPROVED_RATIO.merge(PREV_TOTAL_APPLICATIONS, on='SK_ID_CURR', how='left')
PREV_APPROVED_RATIO['PREV_APPROVED_RATIO'] = PREV_APPROVED_RATIO['PREV_APPROVED_COUNT'] / PREV_APPROVED_RATIO['PREV_TOTAL_APPLICATIONS']
PREV_APPROVED_RATIO.fillna(0, inplace=True)

PREV_AVG_AMT_CREDIT = previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'] \
    .mean().reset_index(name='PREV_AVG_AMT_CREDIT')

result = PREV_TOTAL_APPLICATIONS \
    .merge(PREV_APPROVED_RATIO[['SK_ID_CURR', 'PREV_APPROVED_RATIO']], on='SK_ID_CURR', how='left') \
    .merge(PREV_AVG_AMT_CREDIT, on='SK_ID_CURR', how='left')

result.fillna(0, inplace=True)
result.head()

,SK_ID_CURR,PREV_TOTAL_APPLICATIONS,PREV_APPROVED_RATIO,PREV_AVG_AMT_CREDIT
0,100001,1,1.0,23787.00
1,100002,1,1.0,179055.00
2,100003,3,1.0,484191.00
3,100004,1,1.0,20106.00
4,100005,2,0.5,20076.75


In [15]:
app_test_new = app_test_new.merge(result, on='SK_ID_CURR')
app_test_new.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,ACTIVE_CREDIT_RATIO,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS,LATE_PAYMENTS_COUNT,AVG_LATE_DAYS,LATE_PAYMENT_RATIO,PREV_TOTAL_APPLICATIONS,PREV_APPROVED_RATIO,PREV_AVG_AMT_CREDIT
0,100013,Cash loans,M,Working,Higher education,House / apartment,1,0,2,2,...,0.000000,1,0.115301,96.0,11,7.636364,0.070968,4,0.750000,146134.12500
1,100042,Cash loans,F,State servant,Secondary / secondary special,House / apartment,1,0,2,2,...,0.571429,2,0.370624,84.0,4,8.750000,0.021978,9,1.000000,458168.00000
2,100067,Cash loans,F,Working,Higher education,House / apartment,1,1,2,2,...,0.294118,3,0.604061,87.0,6,3.333333,0.037267,23,0.304348,189582.26087
3,100107,Cash loans,M,Working,Secondary / secondary special,House / apartment,1,0,1,1,...,1.000000,1,0.665672,11.0,3,17.333333,0.075000,2,1.000000,56094.75000
4,100109,Cash loans,F,Working,Secondary / secondary special,House / apartment,1,1,2,2,...,0.230769,3,0.283998,72.0,3,5.000000,0.034091,9,0.333333,177126.00000


In [16]:
app_test_new['YEARS_BIRTH'] = (-app_test_new['DAYS_BIRTH'] / 365).astype(int)

In [29]:
filter_columns(app_test_new, columns_to_keep)

Semua kolom tersedia dalam data.


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,AVG_LATE_DAYS,LATE_PAYMENT_RATIO,PREV_TOTAL_APPLICATIONS,PREV_APPROVED_RATIO,PREV_AVG_AMT_CREDIT,YEARS_BIRTH,CNT_FAM_MEMBERS,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,CNT_CHILDREN
0,100013,0,1,3,1,1,1,0,2,2,...,7.636364,0.070968,4,0.750000,146134.125000,54,2.0,0.0,0.0,0
1,100042,0,0,2,4,1,1,0,2,2,...,8.750000,0.021978,9,1.000000,458168.000000,50,2.0,0.0,0.0,0
2,100067,0,0,3,1,1,1,1,2,2,...,3.333333,0.037267,23,0.304348,189582.260870,28,3.0,0.0,0.0,1
3,100107,0,1,3,4,1,1,0,1,1,...,17.333333,0.075000,2,1.000000,56094.750000,37,2.0,0.0,0.0,0
4,100109,0,0,3,4,1,1,1,2,2,...,5.000000,0.034091,9,0.333333,177126.000000,53,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,455378,0,0,2,4,1,1,0,2,2,...,4.250000,0.030769,5,1.000000,103576.500000,48,2.0,0.0,0.0,0
2391,455436,0,0,3,3,1,1,1,1,1,...,8.000000,0.038462,2,1.000000,77701.500000,41,2.0,0.0,0.0,0
2392,455648,0,0,0,1,1,1,0,2,2,...,7.600000,0.027624,6,0.666667,243972.750000,37,3.0,1.0,1.0,1
2393,455863,0,0,1,1,1,0,0,2,2,...,9.200000,0.054348,7,0.428571,306642.214286,57,2.0,0.0,0.0,0


In [18]:
app_test_new.select_dtypes(include=['object', 'category']).columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'ORGANIZATION_TYPE'],
      dtype='object')

In [19]:
le = LabelEncoder()
string_columns = app_test_new.select_dtypes(include=['object', 'category']).columns

label_mappings = {}

for column in string_columns:
    app_test_new[column] = le.fit_transform(app_test_new[column])
    label_mappings[column] = dict(enumerate(le.classes_))

app_test_new.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS,LATE_PAYMENTS_COUNT,AVG_LATE_DAYS,LATE_PAYMENT_RATIO,PREV_TOTAL_APPLICATIONS,PREV_APPROVED_RATIO,PREV_AVG_AMT_CREDIT,YEARS_BIRTH
0,100013,0,1,3,1,1,1,0,2,2,...,1,0.115301,96.0,11,7.636364,0.070968,4,0.750000,146134.12500,54
1,100042,0,0,2,4,1,1,0,2,2,...,2,0.370624,84.0,4,8.750000,0.021978,9,1.000000,458168.00000,50
2,100067,0,0,3,1,1,1,1,2,2,...,3,0.604061,87.0,6,3.333333,0.037267,23,0.304348,189582.26087,28
3,100107,0,1,3,4,1,1,0,1,1,...,1,0.665672,11.0,3,17.333333,0.075000,2,1.000000,56094.75000,37
4,100109,0,0,3,4,1,1,1,2,2,...,3,0.283998,72.0,3,5.000000,0.034091,9,0.333333,177126.00000,53


In [20]:
rows_with_inf = np.isinf(app_test_new).any(axis=1)  
rows_with_nan = app_test_new[app_test_new.isnull().any(axis=1)]
print(f"Jumlah baris dengan infinity: {rows_with_inf.sum()}")
print(f"Jumlah baris dengan NaN: {rows_with_nan.shape[0]}")

Jumlah baris dengan infinity: 2
Jumlah baris dengan NaN: 3


In [21]:
app_test_new = app_test_new.dropna(axis=0)
app_test_new = app_test_new[~np.isinf(app_test_new).any(axis=1)]  # Hapus baris dengan inf

In [38]:
app_test_new.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,...,TOTAL_MONTHS_DPD,AVG_CREDIT_LIMIT_USAGE,ACTIVE_CREDIT_CARDS,LATE_PAYMENTS_COUNT,AVG_LATE_DAYS,LATE_PAYMENT_RATIO,PREV_TOTAL_APPLICATIONS,PREV_APPROVED_RATIO,PREV_AVG_AMT_CREDIT,YEARS_BIRTH
0,100013,0,1,3,1,1,1,0,2,2,...,1,0.115301,96.0,11,7.636364,0.070968,4,0.750000,146134.12500,54
1,100042,0,0,2,4,1,1,0,2,2,...,2,0.370624,84.0,4,8.750000,0.021978,9,1.000000,458168.00000,50
2,100067,0,0,3,1,1,1,1,2,2,...,3,0.604061,87.0,6,3.333333,0.037267,23,0.304348,189582.26087,28
3,100107,0,1,3,4,1,1,0,1,1,...,1,0.665672,11.0,3,17.333333,0.075000,2,1.000000,56094.75000,37
4,100109,0,0,3,4,1,1,1,2,2,...,3,0.283998,72.0,3,5.000000,0.034091,9,0.333333,177126.00000,53


In [22]:
rows_with_inf = np.isinf(app_test_new).any(axis=1)  
rows_with_nan = app_test_new[app_test_new.isnull().any(axis=1)]
print(f"Jumlah baris dengan infinity: {rows_with_inf.sum()}")
print(f"Jumlah baris dengan NaN: {rows_with_nan.shape[0]}")

Jumlah baris dengan infinity: 0
Jumlah baris dengan NaN: 0


In [40]:
scaler = StandardScaler()
app_test_scaled = app_test_new.drop(columns='SK_ID_CURR')
app_test_scaled = scaler.fit_transform(app_test_scaled)

In [23]:
loaded_model = joblib.load('model.pkl')

In [41]:
y_pred = loaded_model.predict(app_test_scaled)

In [42]:
y_pred.shape, app_test_new.shape

((2390,), (2390, 48))

In [43]:
sk_id_curr = app_test_new['SK_ID_CURR']
target = y_pred
submission = pd.DataFrame({
    'SK_ID_CURR': sk_id_curr,
    'TARGET': target
})

submission.to_csv('output/submission.csv', index=False)